<a href="https://colab.research.google.com/github/gauravreddy08/learning-pytorch/blob/main/05_pytorch_modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05 | Going **Modular** with PyTorch
https://www.learnpytorch.io/05_pytorch_going_modular/

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn

In [2]:
import os 

os.mkdir("going_modular")

In [3]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for 
image classification data.
"""
import torch
import os
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

NUM_WORKERS = os.cpu_count()

def create_dataset(
            train_dir: str, test_dir: str,
            transform: transforms.Compose, 
            batch_size: int = 32,
            num_workers: int = NUM_WORKERS
          ):
  """
  Creates torch dataloaders from the given train and test directories
  for image classification tasks.

  Args: 
    train_dir: Path of training directory.
    test_dir: Path of testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch (default=32).
    num_works: An integer for number of workers per DataLoader.
  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
  """
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  class_names = train_data.classes

  train_dataloader = torch.utils.data.DataLoader(
      train_data, batch_size=batch_size,
      shuffle=True, num_workers=num_workers,
      pin_memory=True
  )
  test_dataloader = torch.utils.data.DataLoader(
      test_data, batch_size=batch_size,
      shuffle=False, num_workers=num_workers,
      pin_memory=True
  )

  return train_dataloader, test_dataloader, class_names

Writing going_modular/data_setup.py


In [13]:
%%writefile going_modular/models.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""

import torch
from torch import nn

class TinyVGG(nn.Module):
  """
  Creates TinyVGG architecture.

  Args:
    input_shape: Number of input channels.
    hidden_units: Number of hidden units (neurons) between the layers.
    output_shape: Number of output units (number of classes).
  """
  def __init__(self, input_shape: int,
             hidden_units: int,
             output_shape: int):
    super().__init__()

    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3, padding=0, stride=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3, padding=0, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, 3, 1, 0),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, 3, 1, 0),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Overwriting going_modular/models.py


In [14]:
from going_modular.models import TinyVGG 

model = TinyVGG(2, 10, 5)
model

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(2, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=5, bias=True)
  )
)

In [17]:
%%writefile going_modular/engine.py
import torch 
from torch import nn
from typing import Dict, List, Tuple
from sklearn.metrics import accuracy_score

def train_step(model: nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: nn.Module, optimizer: torch.optim.Optimizer,
               device: torch.device='cuda' if torch.cuda.is_available() else 'cpu'):
  """
  Training Torch model and updates weights per batch.
  
  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    Tuple(train_loss, train_accuracy)
  """

  train_loss = 0
  train_acc = 0
  
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    preds = model(X)
    loss = loss_fn(preds, y)
    
    train_loss+=loss
    train_acc += accuracy_score(y.cpu(), torch.argmax(torch.softmax(preds, dim=1), dim=1).cpu())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  return train_loss/len(dataloader), train_acc/len(dataloader)

def test_step(model: nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: nn.Module,
              device: torch.device='cuda' if torch.cuda.is_available() else 'cpu'):
  """
  Runs Torch model in evaluation model.
  
  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    Tuple(test_loss, test_accuracy)
  """
  test_loss = 0
  test_acc = 0

  from sklearn.metrics import accuracy_score
  
  model.eval()
  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      preds = model(X)
      test_loss += loss_fn(preds, y)
      test_acc += accuracy_score(y.cpu(), torch.argmax(torch.softmax(preds, dim=1), dim=1).cpu())
    return test_loss/len(dataloader), test_acc/len(dataloader)

def train(model: nn.Module, epochs: int,
          train_data: torch.utils.data.DataLoader,
          loss: nn.Module, optimizer: torch.optim.Optimizer,
          test_data: torch.utils.data.DataLoader, 
          device: torch.device='cuda' if torch.cuda.is_available() else 'cpu'):
  
  """
  Trains and tests a PyTorch model.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {epochs: []),
                  train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]} 
  """
  results = {'epochs': list(range(epochs)),
             'train_loss': [],
             'train_acc': [],
             'test_loss': [],
             'test_acc': []}

  for epoch in range(epochs):
    print(f"EPOCH [{epoch}]")
    train_loss, train_acc = train_step(model, train_data,
                                       loss, optimizer)
    
    test_loss, test_acc = test_step(model, test_data, loss)

    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}% \n------------")
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  return results


Writing going_modular/engine.py


In [49]:
%%writefile going_modular/train.py
print(f"[INFO] Installing Dependencies...")
import argparse
import pip
parser = argparse.ArgumentParser()

parser.add_argument("--train", type=str)
parser.add_argument("--test", type=str)
parser.add_argument("--epochs", type=int, default=5)
parser.add_argument("--batch", type=int, default=32)
parser.add_argument("--hidden", type=int, default=10)
parser.add_argument("--lr", type=float, default=0.1)

args = parser.parse_args()

import os
import torch 
from torch import nn
from torchvision import transforms

from datetime import datetime

try: 
  import torchinfo
except:
  pip.main(['install', 'torchinfo'])       
  import torchinfo

import data_setup, engine, models
from torchinfo import summary

EPOCHS = args.epochs
BATCH_SIZE = args.batch
HIDDEN_UNITS = args.hidden
LEARNING_RATE = args.lr
LINE_BR = "-"*90

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('\n')
print(LINE_BR)
print(f"[Hyperparameters]: ")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Hidden Units: {HIDDEN_UNITS}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"CUDA Available = {torch.cuda.is_available()} | Device: {DEVICE}")
print(LINE_BR)

train_dir = args.train
test_dir = args.test

print(f"[INFO] Training Directory: {train_dir}")
print(f"[INFO] Testing Directory: {test_dir}")
print(LINE_BR)

data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataset(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

print(f"[INFO] Model Initialised...")

model = models.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(DEVICE)

summary(model, input_size=[1, 3, 64, 64])

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)
print(LINE_BR)
print(f"[INFO] Training...")
# Start training with help from engine.py
engine.train(model=model,
             train_data=train_dataloader,
             test_data=test_dataloader,
             loss=loss_fn,
             optimizer=optimizer,
             epochs=EPOCHS,
             device=DEVICE)

name = "model_" + datetime.now().strftime("%d%M%y_%H%M%S")+ '.pth'
torch.save(obj=model.state_dict(), f=name)
print(f"[INFO] Saved model as {name}")
print(LINE_BR)

Overwriting going_modular/train.py


---
---
## Training the model with only a `single` line

### Downloading the **data**

In [ ]:
import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...") 
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

#### Here we go...

In [50]:
!python going_modular/train.py --train '/content/data/pizza_steak_sushi/train' --test '/content/data/pizza_steak_sushi/test' \
                               --epochs 5 --batch 32 --hidden 10 --lr 0.1

[INFO] Installing Dependencies...


------------------------------------------------------------------------------------------
[Hyperparameters]: 
Epochs: 5
Batch Size: 32
Hidden Units: 10
Learning Rate: 0.1
CUDA Available = True | Device: cuda
------------------------------------------------------------------------------------------
[INFO] Training Directory: /content/data/pizza_steak_sushi/train
[INFO] Testing Directory: /content/data/pizza_steak_sushi/test
------------------------------------------------------------------------------------------
[INFO] Model Initialised...
------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [1, 3]                    --
├─Sequential: 1-1                        [1, 10, 30, 30]           --
│    └─Conv2d: 2-1                       [1, 10, 62, 62]           280
│    └─ReLU: 2-2                        